# LOAD DATA

In [1]:
import pandas as pd
import optuna
import numpy as np
import faiss
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
import ast

d:\Mestrado\embedding_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataframe = pd.read_csv('data/xiami/dataframe.csv')

In [3]:
dataframe['song_ids_order'] = dataframe['song_ids_order'].apply(ast.literal_eval)

In [4]:
# min 2 itens para input
# 1 item para recomendacao
# 1 item para avaliacao

dataframe = dataframe[dataframe['length'] > 3]

In [5]:
dataframe['last_item'] = dataframe['song_ids_order'].apply(lambda x: x.pop() if len(x) > 0 else None)

In [6]:
all_songs = set([item for sublist in dataframe['song_ids_order'] for item in sublist])

In [7]:
dataframe = dataframe[dataframe['last_item'].isin(all_songs)]

In [8]:
sequences = dataframe['song_ids_order']

# MODEL

In [9]:
from gensim.models.callbacks import CallbackAny2Vec

class EpochLogger(CallbackAny2Vec):
    '''Callback para registrar a perda após cada época.'''
    def __init__(self):
        self.epoch = 0
        self.loss_previous_step = 0
        self.losses = []

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_previous_step
        self.loss_previous_step = loss
        self.losses.append(loss_now)
        print(f'Perda após época {self.epoch}: {loss_now}')
        self.epoch += 1

# epoch_logger = EpochLogger()

class EarlyStoppingCallback(CallbackAny2Vec):
    '''Callback para early stopping baseado em uma melhoria percentual na perda.'''
    def __init__(self, epoch_logger, patience=3, min_percent_improvement=0.01):
        self.epoch_logger = epoch_logger
        self.patience = patience  # número de épocas sem melhoria
        self.min_percent_improvement = min_percent_improvement  # percentual mínimo de melhoria
        self.best_loss = float('inf')  # inicializar com um valor alto
        self.counter = 0  # contar épocas sem melhoria

    def on_epoch_end(self, model):
        loss_now = self.epoch_logger.losses[-1]  # Obter a última perda registrada pelo EpochLogger
        
        # Checar se houve uma melhoria percentual significativa
        if self.best_loss == float('inf'):
            improvement = float('inf')
        else:
            improvement = (self.best_loss - loss_now) / self.best_loss

        print(f'Melhoria percentual: {improvement * 100:.2f}%')

        if improvement > self.min_percent_improvement:
            self.best_loss = loss_now
            self.counter = 0  # Resetar o contador de épocas sem melhoria
        else:
            self.counter += 1
            print(f'Early stopping counter: {self.counter}/{self.patience}')
            
            # Parar se o número de épocas sem melhoria exceder a paciência
            if self.counter >= self.patience:
                print(f'Early stopping ativado na época {self.epoch_logger.epoch}')
                model.running_training = False  # Isso interrompe o treinamento

# early_stopping = EarlyStoppingCallback(epoch_logger=epoch_logger, patience=3, min_percent_improvement=0.01)

In [10]:
from gensim.models import Word2Vec

def objective(trial):

    epoch_logger = EpochLogger()
    early_stopping = EarlyStoppingCallback(epoch_logger=epoch_logger, patience=3, min_percent_improvement=0.01)

    # Hiperparâmetros que serão ajustados
    vector_size = trial.suggest_categorical('vector_size', [64, 128, 256, 512])
    window = trial.suggest_int('window', 3, 10)
    alpha = trial.suggest_float('alpha', 1e-4, 1e-1, log=True)
    min_alpha = trial.suggest_float('min_alpha', 1e-4, 1e-2, log=True)
    negative = trial.suggest_int('negative', 5, 20)
    
    # Treinamento do modelo Word2Vec
    model = Word2Vec(
        sequences,
        vector_size=vector_size,
        window=window,
        min_count=1,
        workers=4,
        sg=1,
        negative=negative,
        epochs=20,
        sample=0.001,
        alpha=alpha,
        min_alpha=min_alpha,
        compute_loss=True,
        callbacks=[epoch_logger, early_stopping]  # Callback para log
    )

    # Processo para identificar o score final (substitua pelo seu método)
    score = calcular_score_final(model, dataframe)
    
    return score

In [11]:
def get_average_embedding(song_ids, model):
    embeddings = []
    for song_id in song_ids:
        song_id_str = str(song_id)
        if song_id_str in model.wv:
            embeddings.append(model.wv[song_id_str])
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

In [12]:
def check_last_item_in_similar(row):
    last_item = str(row['last_item'])
    similar_ids = row['similar_song_ids']
    return last_item in similar_ids

In [13]:
def calcular_score_final(model, dataframe):
    # Gerar embeddings médios
    dataframe['average_embedding'] = dataframe['song_ids_order'].apply(lambda x: get_average_embedding(x, model))
    
    # Preparar embeddings para comparação
    song_ids = list(model.wv.index_to_key)
    embeddings = np.array([model.wv[song_id] for song_id in song_ids]).astype('float32')
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)  # Usando Inner Product para similaridade cosseno
    index.add(embeddings)
    
    average_embeddings = np.stack(dataframe['average_embedding'].values).astype('float32')
    average_embeddings = average_embeddings / np.linalg.norm(average_embeddings, axis=1, keepdims=True)
    
    k = 10
    distances, indices = index.search(average_embeddings, k)
    similar_song_ids = [[song_ids[idx] for idx in neighbors] for neighbors in indices]
    dataframe['similar_song_ids'] = similar_song_ids
    
    dataframe['is_last_item_in_similar'] = dataframe.apply(check_last_item_in_similar, axis=1)

    counts = dataframe.groupby('is_last_item_in_similar').size()

    if True in counts:
        # Calcular a porcentagem de True
        percentage_true = (counts[True] / counts.sum()) * 100
    else:
        # Se não houver nenhum True, a porcentagem é 0
        percentage_true = 0

    return percentage_true

# FIND BEST PARAMETERS

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)

print("Melhores hiperparâmetros: ", study.best_params)

[I 2024-10-01 21:36:12,469] A new study created in memory with name: no-name-2bd4a8f7-e36d-45aa-bd12-1d80ab6ed667


Perda após época 0: 16777870.0
Melhoria percentual: inf%
Perda após época 1: 287274.0
Melhoria percentual: 98.29%
Perda após época 2: 2179998.0
Melhoria percentual: -658.86%
Early stopping counter: 1/3
Perda após época 3: 3584254.0
Melhoria percentual: -1147.68%
Early stopping counter: 2/3
Perda após época 4: 4144092.0
Melhoria percentual: -1342.56%
Early stopping counter: 3/3
Early stopping ativado na época 5
Perda após época 5: 4488724.0
Melhoria percentual: -1462.52%
Early stopping counter: 4/3
Early stopping ativado na época 6
Perda após época 6: 3204336.0
Melhoria percentual: -1015.43%
Early stopping counter: 5/3
Early stopping ativado na época 7
Perda após época 7: 2105536.0
Melhoria percentual: -632.94%
Early stopping counter: 6/3
Early stopping ativado na época 8
Perda após época 8: 2174356.0
Melhoria percentual: -656.89%
Early stopping counter: 7/3
Early stopping ativado na época 9
Perda após época 9: 2193588.0
Melhoria percentual: -663.59%
Early stopping counter: 8/3
Early st

[I 2024-10-01 21:46:04,173] Trial 0 finished with value: 18.499852960679956 and parameters: {'vector_size': 128, 'window': 3, 'alpha': 0.0008029676977200852, 'min_alpha': 0.002164588362090582, 'negative': 7}. Best is trial 0 with value: 18.499852960679956.


Perda após época 0: 7424664.5
Melhoria percentual: inf%
Perda após época 1: 4664341.5
Melhoria percentual: 37.18%
Perda após época 2: 3853785.0
Melhoria percentual: 17.38%
Perda após época 3: 3139513.0
Melhoria percentual: 18.53%
Perda após época 4: 2720210.0
Melhoria percentual: 13.36%
Perda após época 5: 2545284.0
Melhoria percentual: 6.43%
Perda após época 6: 2427958.0
Melhoria percentual: 4.61%
Perda após época 7: 2305262.0
Melhoria percentual: 5.05%
Perda após época 8: 2235068.0
Melhoria percentual: 3.04%
Perda após época 9: 2188452.0
Melhoria percentual: 2.09%
Perda após época 10: 1633586.0
Melhoria percentual: 25.35%
Perda após época 11: 1514016.0
Melhoria percentual: 7.32%
Perda após época 12: 1532692.0
Melhoria percentual: -1.23%
Early stopping counter: 1/3
Perda após época 13: 1490204.0
Melhoria percentual: 1.57%
Perda após época 14: 1441612.0
Melhoria percentual: 3.26%
Perda após época 15: 1474328.0
Melhoria percentual: -2.27%
Early stopping counter: 1/3
Perda após época 16:

[I 2024-10-01 21:59:07,413] Trial 1 finished with value: 23.477854725151797 and parameters: {'vector_size': 128, 'window': 4, 'alpha': 0.09543064176073654, 'min_alpha': 0.0010227024711055926, 'negative': 12}. Best is trial 1 with value: 23.477854725151797.


Melhores hiperparâmetros:  {'vector_size': 128, 'window': 4, 'alpha': 0.09543064176073654, 'min_alpha': 0.0010227024711055926, 'negative': 12}
